# Cirq - t|ket> routing example

First import cirq and required classes and functions from t|ket>.
`SquareGrid` is the t|ket> class representing a rectangular grid of qubits, with qubits indexed continuously from (0,0), e.g. for a 3x3 grid, the indexing is: 

0 1 2 <br>
3 4 5 <br>
6 7 8

`SquareGrid` can be used to generate XmonDevice objects corresponding to rectangular grids. 

`route` will take our circuit and return a circuit routed on the provided XmonDevice. Routing involves optimally adding swap commands such that the necessary 2-qubit interactions only take place between nearest-neighbour qubits (as is normally the case with superconducting qubits). It also returns a pair of maps representing the qubit permutations specifying locations of each qubit on nodes of the architecture at the start and end of the circuit. <br>

`get_grid_qubits` is a helper function to return a list of cirq.GridQubit which correspond to a list of provided indices on the given SquareGrid.

In [1]:
import cirq

from pytket._routing import SquareGrid, route
from pytket.cirq import get_grid_qubits, cirq_to_tk, tk_to_cirq

Let's initialize a 3x3 grid and get the corresponding 9 qubits. This will be useful later on for translating qubits back to their locations on this grid.

In [2]:
arc = SquareGrid(3, 3)
qubits = get_grid_qubits(arc, range(9))

Generate an arbitrary sample circuit. See the [readme](https://github.com/CQCL/pytket/blob/master/README.md) for currently supported gates, more added soon!

In [3]:
circuit = cirq.Circuit.from_ops(
        cirq.H(qubits[0]),
        cirq.X(qubits[1]),
        cirq.CNOT(qubits[0], qubits[4]),
        cirq.Y(qubits[2]),
        cirq.CNOT(qubits[2], qubits[4]),
        cirq.CNOT(qubits[3], qubits[4]),
        cirq.Y(qubits[3]),
        cirq.CNOT(qubits[3], qubits[5]),
        cirq.Z(qubits[4])
    )
circuit

┌──┐             ┌──┐
(0, 0): ───H───@─────────────────────
               │
(0, 1): ───X───┼─────────────────────
               │
(0, 2): ───────┼Y───@────────────────
               │    │
(1, 0): ───────┼────┼───@───Y───@────
               │    │   │       │
(1, 1): ───────X────X───X───────┼Z───
                                │
(1, 2): ────────────────────────X────
              └──┘             └──┘

And now we can route the circuit on the architecture, subject to the constraint that the 2-qubit interactions can only occur between neighbouring qubits. We start by converting it to a tket Circuit object using `cirq_to_tk`.

Tket Circuits are device-independent, so the qubit identifiers are not associated to nodes on a given architecture. The routed circuit will always have the same mapping of input nodes to qubits as in the original circuit. However, the routing process gives us a map saying how to position the qubits on our device. The conversion `tk_to_circ` takes a map from the tket Circuit qubits to the corresponding `QubitId` objects.

In [4]:
tk_circ = cirq_to_tk(circuit)
tk_routed, qmap = route(tk_circ, arc)
print(qmap)
#tk_routed.reorderCircuitBoundary(qmap[0])
final_qubits = [qubits[j] for i,j in enumerate(qmap[0])]
tk_to_cirq(tk_routed, final_qubits)

([8, 3, 4, 6, 5, 7, 2, 1, 0], [8, 3, 5, 7, 4, 6, 2, 1, 0])


┌──┐
(1, 0): ───X────────────────────────────────────

(1, 1): ───Y────────@───X───@───X───X───Z───────
                    │   │   │   │   │
(1, 2): ────────X───X───@───X───@───┼───────────
                │                   │
(2, 0): ───X───@┼───X───────────────┼───────X───
           │   ││   │               │       │
(2, 1): ───@───X┼───@───────────────@───Y───@───
                │
(2, 2): ───H────@───────────────────────────────
              └──┘

Note that the output qubits do not necessarily align with the original qubits as we have not swapped them back. The other qubit map returned after routing (`qmap[1]`) maps from the tket index of a qubit in the original circuit to its position at the end of the routed circuit.

Two swaps were needed to route this particular circuit. Can it be done with fewer?